In [164]:
from Crypto.Hash import SHAKE256

In [71]:
Zx.<x> = ZZ[]

k = 2
n = 2^k
q = 12 * 1024 + 1
phi = x^n + 1
sigma = 1.17 / sqrt(q / (2*n))   
RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]

In [49]:
def balance(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return Zx(g)

In [50]:
def merge(a,b,n):
    a = a.subs(x=x^2)
    b = x*b.subs(x=x^2)
    return a + b 

In [51]:
def split(f,n):
    f0 = list(f[2*i] for i in range(n//2+1))
    f1 = list(f[2*i+1] for i in range(n//2+1))
    return Zx(f0), Zx(f1)

In [52]:
def InnerProduct(a,b,n):
    s = [a[i]*b[i] for i in range(n)]
    return sum(s)

In [53]:
def EuclideanNorm(a,n):
    b = InnerProduct(a,a,n)
    return sqrt(float(b))

In [54]:
def FieldNorm(f, n):
    f0, f1 = split(f,n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n//2)+1)

In [55]:
def HermitianAdjointPoly(p, n):
    f=[p[0]]
    for i in range(1,n):    
        f.append(-p[n-i])
    return Zx(f) 

In [56]:
from os import urandom

LN2 = 0.69314718056

def UniformBits(k):
    return int.from_bytes(bytes(list(floor(uniform(0, 256)) for i in range(k / 8))), 'little')

def BaseSampler():
    u = UniformBits(72)
    z_0 = 0
    RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]
    for i in range(0, 18):
        z_0 = z_0 + int(u<RCDT[i]) 
    return z_0

def ApproxExp(x, ccs):
    C = [0x00000004741183A3,0x00000036548CFC06,0x0000024FDCBF140A,0x0000171D939DE045,0x0000D00CF58F6F84,0x000680681CF796E3,0x002D82D8305B0FEA,0x011111110E066FD0,0x0555555555070F00,0x155555555581FF00,0x400000000002B400,0x7FFFFFFFFFFF4800,0x8000000000000000]
    y = C[0]
    z = floor(2^63*x)
    for i in range(1, 13):
        y = C[i] - (z*y) >> 63
    z = floor(2^63*ccs)
    y = (z*y) >> 63
    return y

def BerExp(x, ccs):
    s = floor(x/LN2)
    r = x - s*LN2
    s = min(s, 63)
    z = (2*ApproxExp(r, ccs) - 1) >> s
    for i in range(56, -8, -8):
        p = UniformBits(8)
        w = p - ((z >> i) & 0xFF)
        if int(w) == 0:
            break
    return int(w < 0)

def SamplerZ(mu, sigma, sigmamin, sigmamax):
    r = mu - int(floor(mu))
    ccs = sigmamin/sigma
    while True:
        z_0 = BaseSampler()
        b = UniformBits(8)&0x1
        z = b + (2*b-1)*z_0
        x = (z-r)^2/2/sigma^2 - z_0^2/2/sigmamax^2
        if BerExp(x, ccs) == 1:
            return z + int(floor(mu))

 

In [68]:
# ======================== Lattice Matrices ============================

# author: Evgen Postulga
def CyclicRotate(input, n):
    #return input[(len(input)-n):]+input[:(len(input)-n)]
    return input[-n:] + input[0:-n]

# author: Evgen Postulga
def PolyToCirculant(p, n):
    M=[]
    k=p.coefficients(sparse=False)
    while len(k)!=n:
        k.append(0)
    for i in range(n):
        m = CyclicRotate(k, i)
        M.append(m)
    return Matrix(M)

# author: Evgen Postulga
def CirculantToPoly(M):
    # Zx.<x> = PolynomialRing(ZZ)
    return Zx(list(M[0]))
    
# author: Evgen Postulga
def PolyToLattice4(p00, p01, p10, p11, n):
    M=[]
    p=[p00.coefficients(sparse=False),p01.coefficients(sparse=False),p10.coefficients(sparse=False),p11.coefficients(sparse=False)]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return M

# author: Evgen Postulga
def LatticeToPoly4(M, n):
    p00=Zx(M[0][:n])
    p01=Zx(M[0][n:])
    p10=Zx(M[n][:n])
    p11=Zx(M[n][n:])
    return p00, p01, p10, p11

# formula 3.7 page 23б author: Evgen Postulga
def HermitianAdjointMatrix(M, n2):
    a, c, b, d = LatticeToPoly4(M, n2/2)
    a, c, b, d = HermitianAdjointPoly(a, n2/2), HermitianAdjointPoly(c, n2/2),HermitianAdjointPoly(b, n2/2),HermitianAdjointPoly(d, n2/2)
    return PolyToLattice4(a, c, b, d, n2/2)

In [58]:
def Reduce(f, g, F, G, n):
    
    T = Zx.change_ring(QQ).quotient(x^n+1) 
    
    iks = f.parent()([0, 1])
    f_star = HermitianAdjointPoly(f, n)
    g_star = HermitianAdjointPoly(g, n)
    while True:
        num = F*f_star + G*g_star
        num = T(num)
        den = f*f_star + g*g_star
        den = 1 / T(den)
        res = num * den
        k = Zx([int(round(elt)) for elt in res])
        F = F - k*f 
        G = G - k*g
        if all(elt == 0 for elt in k):
            break
    return f, g, F, G

In [59]:
def NTT(f, n, q):
    # Zp
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans

In [60]:
def NTRUSolve(f, g, n, q):
    if n == 1:
        # u, v are numbers
        gcd_, u, v = xgcd(f[0], g[0])
        # print("gcd", u * f + v * g, "\n")
        if gcd_ != 1:
            return None, None, False
        F, G = -v*q, u*q
        # print("F1, G1", F / q, G / q)
        return F, G, True
    else:
        # ▷ f′, g′, F′, G′ ∈ Z[x]/(x^n/2 + 1)
        # ▷ N as defined in either (3.25) or (3.26)
        f_ = FieldNorm(f, n) 
        g_ = FieldNorm(g, n) 
        # print(n//2, f_, g_, sep="\n")
        F_, G_, flag = NTRUSolve(f_, g_, n//2, q)
        if flag:
            F = F_.subs(x=x^2) * g.subs(x=-x) 
            G = G_.subs(x=x^2) * f.subs(x=-x)
            # print("F, G", F, G, sep="\n")
            f, g, F, G = Reduce(f, g, F, G, n)
            return F % (x^n +1), G % (x^n +1), flag
        else:
            return F_, G_, flag
            
        

In [61]:
def NTRUGen(q, n):
    
    # sigma = 1.17 / sqrt(q / (2*n)) 
    while True:

        f = -x^7 + 3*x^6 - x^4 + 4*x^3 + 6*x^2 - 2*x - 4
        g = x^7 - x^6 - 2*x^5 - 4*x^3 - 3*x^2 - x + 7
        
        if gs_norm(f, g, q, n) > (1.17 ** 2) * q:
            print("restart")

        print(f, g, sep="\n")
                
        F, G, flag = NTRUSolve(f, g, n, q)
        F, G = F % (x^n +1), G % (x^n +1)
        
        print("(f*G - g*F) % (x^n + 1) == q: ", (f*G - g*F) % (x^n + 1) == q)
        
        if not flag:
            continue
        else:
            F = Zx([int(coef) for coef in F.coefficients(sparse=False)])
            G = Zx([int(coef) for coef in G.coefficients(sparse=False)])
            break
            
    return f, g, F, G
    

NTRUGen(17, 8)

-x^7 + 3*x^6 - x^4 + 4*x^3 + 6*x^2 - 2*x - 4
x^7 - x^6 - 2*x^5 - 4*x^3 - 3*x^2 - x + 7
(f*G - g*F) % (x^n + 1) == q:  True


(-x^7 + 3*x^6 - x^4 + 4*x^3 + 6*x^2 - 2*x - 4,
 x^7 - x^6 - 2*x^5 - 4*x^3 - 3*x^2 - x + 7,
 2*x^7 + x^6 + 2*x^5 - x^4 - 5*x^3 + 2*x - 5,
 -4*x^7 - x^6 - x^5 + x^4 + 3*x^3 + 2*x^2 - 2*x + 3)

In [140]:
def NTRUGen(q, n):
    
    def gen_poly(n, q):
        
        def D(mu=0):
            z = 0
            for i in range(1, 4096/n + 1):
                sigma_star = 1.17 * sqrt(q / 8192)
                sigmamin, sigmamax = 1.277833697, 1.8205
                zi = SamplerZ(mu, sigma_star, sigmamin, sigmamax)
                z += zi
            return z

        f = [0] * n
        g = [0] * n
        for i in range(n):
            f[i] = D()
            g[i] = D()
        f = Zx(f) % phi
        g = Zx(g) % phi
        return f, g
    
    def gs_norm(f, g, q, n):
        T = Zx.change_ring(QQ).quotient(x^n+1) 
        # Using (3.9) with (3.8) or (3.10)    
        f_star = HermitianAdjointPoly(f, n)
        g_star = HermitianAdjointPoly(g, n)
        first = EuclideanNorm([*g.coefficients(sparse=False), *(-f).coefficients(sparse=False)], n)
        s1 = (q * T(f_star)) / T((f*f_star + g*g_star))
        s2 = (q * T(g_star)) / T((f*f_star + g*g_star))
        second = EuclideanNorm(list(s1) + list(s2), n)
        gamma = max(first, second)
        return gamma
    
    
    while True:
        while True:
            while True:

                f, g = gen_poly(n, q)
                # print(f, g, sep="\n")

                if gs_norm(f, g, q, n) > (1.17 ** 2) * q:
                    # print("restart norm\n")
                    continue
                break

            if  0 in NTT(f, n, q):
                # print("restart ntt\n")
                continue
            break
                
        F, G, flag = NTRUSolve(f, g, n, q)
        
        if not flag:
            # print("restart solve")
            continue
        else:
            F, G = F % (x^n +1), G % (x^n +1)
            F = Zx([int(coef) for coef in F.coefficients(sparse=False)])
            G = Zx([int(coef) for coef in G.coefficients(sparse=False)])
            print("(f*G - g*F) % (x^n + 1) == q", (f*G - g*F) % (x^n + 1) == q)
            break
            
    return f, g, F, G
    

# f, g, F, G = NTRUGen(12 * 1024 + 1, 4)
# print(f, g, F, G, sep="\n")
# print()

# M = PolyToLattice4(g, -f, G, -F, n)
# print(*M, sep="\n")
# print()

# f1, g1, F1, G1 = LatticeToPoly4(M, n)
# print(f1, g1, F1, G1, sep="\n")


In [197]:
def NaiveKeyGen(sigma, q, n):
    
    f, g, F, G = NTRUGen(q, n)
    print(f, g, F, G)
    B = PolyToLattice4(g, -f, G, -F, n)
    sk = B
    print("sk: ", *sk, sep="\n")
    print()
    
    T = Zx.change_ring(Integers(q)).quotient(x^n+1)
    f_q = Zx(lift(1 / T(f))) 
    
    print("f * f_q % phi % q = ", f * f_q % phi % q)
    h = g * f_q % phi % q
    pk = h
    print("pk: ", pk)
    return sk, pk, f, g

sk, pk, f, g = NaiveKeyGen(sigma, q, n)

(f*G - g*F) % (x^n + 1) == q True
331*x^3 + 344*x^2 + 347*x + 288 278*x^3 + 258*x^2 + 314*x + 387 146*x^3 + 440*x^2 + 360*x + 99 44*x^3 + 385*x^2 + 330*x + 167
sk: 
[387, 314, 258, 278, -288, -347, -344, -331]
[278, 387, 314, 258, -331, -288, -347, -344]
[258, 278, 387, 314, -344, -331, -288, -347]
[314, 258, 278, 387, -347, -344, -331, -288]
[167, 330, 385, 44, -99, -360, -440, -146]
[44, 167, 330, 385, -146, -99, -360, -440]
[385, 44, 167, 330, -440, -146, -99, -360]
[330, 385, 44, 167, -360, -440, -146, -99]

f * f_q % phi % q =  1
pk:  3898*x^3 + 728*x^2 + 11927*x + 8437


In [198]:
SALT_LEN = 10

def HashToPoint(salt, message, q, n):

    k = int((2**16) // q)
    
    shake = SHAKE256.new()
    shake.update(salt)
    shake.update(message)
    hashed = [0 for i in range(n)]
    
    i = 0
    j = 0
    while i < n:
        twobytes = shake.read(2)
        elt = (twobytes[0] << 8) + twobytes[1] 
        if elt < k * q:
            hashed[i] = elt % q
            i += 1
        j += 1
    return Zx(hashed) % phi


In [199]:
noise = urandom(SALT_LEN)

def NaiveSign(message, B, q, n):
    
    # list of coefs "c". hash value c ∈ Zq[x]/(ϕ)
    c = HashToPoint(noise, message, q, n)
    print("c ", c)
    
    # type(B) is list
    B = Matrix(B)
    B_inv = B.inverse()
    t = Matrix((c.coefficients(sparse=False) + [0]*n)) * B_inv
    print("t", list(t[0]))
    z = [int(round(float(elt))) for elt in list(t[0])]
    print("z", z)
    # print(t-Matrix(z))
    s = (t-Matrix(z)) * B
    s = list(s[0])
    print("snorm", EuclideanNorm(Zx(s), n))
    print("s", s)
    # ???????? ▷ s1 + s2 * h = c mod (ϕ, q)
    s1 = Zx(s[:n])
    s2 = Zx(s[n:])
    print("sss", s1, s2, sep="\n")
    print()
    print("ver", (s1 + s2 * pk) % phi % q, c, sep="\n")
    return s, s1, s2
    

message = b"message"
s, s1, s2 = NaiveSign(message, sk, q, n)
print("pk * f % q % phi == g: ", pk * f % q % phi == g)


c  7665*x^3 + 2317*x^2 + 10399*x + 1940
t [747840636451299740/12274950433124817, 1138444471068787517/12274950433124817, 769115265444762701/12274950433124817, 941342057857071995/12274950433124817, -1317899320550272330/12274950433124817, -944335691568616021/12274950433124817, -1328794075502417698/12274950433124817, -917805921351629605/12274950433124817]
z [61, 93, 63, 77, -107, -77, -108, -75]
snorm 765.0601283559351
s [-366, -330, -469, -350, 387, 381, 444, 413]
sss
-350*x^3 - 469*x^2 - 330*x - 366
413*x^3 + 444*x^2 + 381*x + 387

ver
9362*x^3 + 6034*x^2 + 10421*x + 8319
7665*x^3 + 2317*x^2 + 10399*x + 1940
pk * f % q % phi == g:  False


In [200]:
def NaiveVerify(message, noise, s2, pk, q, n):
    c = HashToPoint(noise, message, q, n)
    print("c", c)
    s1 = (c - s2 * pk) % phi
    print(s1.change_ring(Integers(q)))
    s1 = balance((c - s2 * pk) % phi % q, q, n) 
    print(s1.coefficients(sparse=False)+s2.coefficients(sparse=False))
    print(EuclideanNorm(s1.coefficients(sparse=False)+s2.coefficients(sparse=False), n))
    if EuclideanNorm(s1.coefficients(sparse=False)+s2.coefficients(sparse=False), n) <= sqrt(208714):
        return True
    else:
        return False
    
    
print(NaiveVerify(message, noise, s2, pk, q, n))
print(NaiveVerify(b"qqqqqqqqqq", noise, s2, pk, q, n))

c 7665*x^3 + 2317*x^2 + 10399*x + 1940
10242*x^3 + 8103*x^2 + 11937*x + 5544
[5544, -352, -4186, -2047, 387, 381, 444, 413]
7250.6996214158535
False
c 2163*x^3 + 1989*x^2 + 2204*x + 2229
4740*x^3 + 7775*x^2 + 3742*x + 5833
[5833, 3742, -4514, 4740, 387, 381, 444, 413]
9532.58878794213
False


In [122]:
print(float(62118221042226931080/1481241117720232013))
print(int(round(float(62118221042226931080/1481241117720232013))))
print(float(-93905902022813466/1481241117720232013))

41.93660323029154
42
-0.06339676970846136


In [ ]:
def NaiveVerify(message, r, s2, pk, q, n):
    pass

In [ ]:
message = b"Hello"

In [93]:
def KeyGen(sigma, q, n):
    f, g, F, G = NTRUGen(q, n)
    B = PolyToLattice4(g, G, -f, -F, n)
    B_ = FFT(B)
    B_adj = HermitianAdjointMatrix(B_, n2)
    G = mult_matrix(B_, B_adj)
    # Computing the LDL⋆ tree
    T = LDL(g00, g01, g10, g11, n)
    
    for leaf in T:
        leaf.value = sigma / sqrt(leaf.value)
        
    sk = (B_, T)
    f_q = invertmodprime(f, q, n)
    h = g * f_q % p
    pk = h
    return sk, pk